# Streaming processing of cosmic rays using Drift Tubes detectors

Dal Magro Matteo

Rosset Lorenzo

Zanola Andrea

## Producer

In [1]:
import boto3
import pandas as pd
import time
import json
import numpy as np
import sys
from kafka import KafkaProducer

### Load Data from S3 bucket

In [2]:
s3_client = boto3.client('s3',
    aws_access_key_id='e682710620e4404a92ceb9beab7e7f1a',
    aws_secret_access_key='b9f491755eea4698a919e10f0db026b8',
    endpoint_url='https://cloud-areapd.pd.infn.it:5210',
    verify=False)

In [3]:
# Retrieve the list of existing buckets
response = s3_client.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

/usr/lib/python3.6/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud-areapd.pd.infn.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Existing buckets:
  2021-07-13_Run1
  LCPmodB-Y3_CMS_FCNC
  MAPD_miniDT_batch
  MAPD_miniDT_stream
  RUN001220
  RUN001221
  RUN001222
  RUN001223
  RUN001231
  RUN001232
  RUN001233
  RUN001234
  RUN001235
  RUN001236
  RUN001237
  RUN001238
  RUN001240
  RUN001242
  Run001213
  description
  mybucket


In [4]:
bucket = 'MAPD_miniDT_stream'
keys = []
for obj in s3_client.list_objects_v2(Bucket=bucket)['Contents']:
    keys.append(obj['Key'])
    
print(keys[:10])

/usr/lib/python3.6/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud-areapd.pd.infn.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


['data_000000.txt', 'data_000001.txt', 'data_000002.txt', 'data_000003.txt', 'data_000004.txt', 'data_000005.txt', 'data_000006.txt', 'data_000007.txt', 'data_000008.txt', 'data_000009.txt']


### Sending Data to topic_stream

In [5]:
KAFKA_BOOTSTRAP_SERVERS = '10.67.22.127:9092'
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS, 
                         linger_ms=200, batch_size=100000)


In [9]:
count = 0
#x=[]
while True:
    try:
        key = keys[count % len(keys)]
        response = s3_client.get_object(Bucket=bucket, Key=key)
        
        #rate = 2000  # ADC rate in rows per second. Tipical is 1000
        body = pd.read_csv(response.get('Body'), dtype='str')
    
        for index, row in body.iterrows():
            producer.send(topic='topic_stream', value=row.to_json().encode('utf-8'))
            #print(row.to_json().encode('utf-8'))
            #time.sleep(1/rate)
        count += 1
    except KeyboardInterrupt:
        break

/usr/lib/python3.6/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud-areapd.pd.infn.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
